# அமர்வு 5 – பல முகவர் ஒருங்கிணைப்பாளர்

Foundry Local பயன்படுத்தி ஒரு எளிய இரண்டு முகவர் குழாய்வழியை (ஆராய்ச்சியாளர் -> தொகுப்பாளர்) விளக்குகிறது.


### விளக்கம்: சார்பு நிறுவல்
உள்ளூர் மாடல் அணுகல் மற்றும் உரையாடல் முடிவுகளுக்கு தேவையான `foundry-local-sdk` மற்றும் `openai` ஐ நிறுவுகிறது. இது மீண்டும் மீண்டும் செயல்படக்கூடியது.


# நிலைமை
குறைந்தபட்ச இரண்டு-முகவர் ஒருங்கிணைப்பு முறைமையை செயல்படுத்துகிறது:
- **ஆராய்ச்சியாளர் முகவர்** சுருக்கமான உண்மையான புள்ளிகளை சேகரிக்கிறார்
- **தொகுப்பாளர் முகவர்** நிர்வாக தெளிவுக்காக மறுபதிவு செய்கிறார்

ஒவ்வொரு முகவருக்கும் பகிரப்பட்ட நினைவகம், இடைநிலை வெளியீட்டை தொடர்ச்சியாக பரிமாறுதல், மற்றும் ஒரு எளிய குழாய் செயல்பாட்டை காட்டுகிறது. மேலும் பாத்திரங்களை (எ.கா., விமர்சகர், சரிபார்ப்பாளர்) அல்லது இணை கிளைகளுக்கு விரிவாக்கக்கூடியது.

**சுற்றுச்சூழல் மாறிகள்:**
- `FOUNDRY_LOCAL_ALIAS` - பயன்படுத்த வேண்டிய இயல்புநிலை மாடல் (இயல்புநிலை: phi-4-mini)
- `AGENT_MODEL_PRIMARY` - முதன்மை முகவர் மாடல் (ALIAS ஐ மீறுகிறது)
- `AGENT_MODEL_EDITOR` - தொகுப்பாளர் முகவர் மாடல் (முதன்மை மாடலின் இயல்புநிலை)

**SDK குறிப்பு:** https://github.com/microsoft/Foundry-Local/tree/main/sdk/python/foundry_local

**இது எப்படி செயல்படுகிறது:**
1. **FoundryLocalManager** தானாகவே Foundry Local சேவையை தொடங்குகிறது
2. குறிப்பிடப்பட்ட மாடலை பதிவிறக்கம் செய்து ஏற்றுகிறது (அல்லது சேமிக்கப்பட்ட பதிப்பை பயன்படுத்துகிறது)
3. தொடர்புக்கு OpenAI-இன் இணக்கமான இறுதிப்புள்ளியை வழங்குகிறது
4. ஒவ்வொரு முகவரும் தனிப்பட்ட பணிகளுக்கு வேறுபட்ட மாடலை பயன்படுத்த முடியும்
5. உள்ளமைக்கப்பட்ட மீண்டும் முயற்சி செய்யும் தரவுகள் தற்காலிக தோல்விகளை மென்மையாக கையாளுகிறது

**முக்கிய அம்சங்கள்:**
- ✅ தானியங்கி சேவை கண்டறிதல் மற்றும் தொடக்கம்
- ✅ மாடல் வாழ்க்கைச்சுழற்சி மேலாண்மை (பதிவிறக்கம், சேமிப்பு, ஏற்றுதல்)
- ✅ OpenAI SDK இணக்கத்தன்மை பரிச்சய API க்காக
- ✅ முகவர் சிறப்பு பணிகளுக்கு பல மாடல் ஆதரவு
- ✅ வலுவான பிழை கையாளுதல் மீண்டும் முயற்சி தரவுகளுடன்
- ✅ உள்ளூர் தீர்வு (மேக API தேவையில்லை)


In [16]:
# Install dependencies
!pip install -q foundry-local-sdk openai

### விளக்கம்: முக்கிய இறக்குமுதல்கள் & டைப்பிங்
Agent செய்தி சேமிப்பிற்கான dataclasses மற்றும் தெளிவிற்கான டைப்பிங் ஹின்ட்ஸ் அறிமுகப்படுத்துகிறது. Foundry Local மேலாளர் + OpenAI கிளையண்டை அடுத்தடுத்த agent செயல்பாடுகளுக்காக இறக்குமதி செய்கிறது.


In [17]:
from dataclasses import dataclass, field
from typing import List
import os
from foundry_local import FoundryLocalManager
from openai import OpenAI

### விளக்கம்: மாடல் தொடக்கத்திற்கான (SDK முறை)
Foundry Local Python SDK-ஐ வலுவான மாடல் மேலாண்மைக்காக பயன்படுத்துகிறது:
- **FoundryLocalManager(alias)** - சேவையை தானாக தொடங்குகிறது மற்றும் alias மூலம் மாடலை ஏற்றுகிறது
- **get_model_info(alias)** - alias ஐ குறிப்பிட்ட மாடல் ID-க்கு தீர்மானிக்கிறது
- **manager.endpoint** - OpenAI client-க்கு சேவை முடுக்கத்தை வழங்குகிறது
- **manager.api_key** - API key-ஐ வழங்குகிறது (உள்ளூர் பயன்பாட்டிற்கு விருப்பமானது)
- வேறுபட்ட முகவர்களுக்கான தனித்த மாடல்களை ஆதரிக்கிறது (முதன்மை vs தொகுப்பாளர்)
- மாறுபாடுகளுக்கு எதிராக உறுதிப்பாட்டுடன் உள்ளமைக்கப்பட்ட மீண்டும் முயற்சி செய்யும் தருக்கம்
- சேவை தயாராக உள்ளதா என்பதை உறுதிப்படுத்த இணைப்பு சரிபார்ப்பு

**முக்கிய SDK முறை:**
```python
manager = FoundryLocalManager(alias)
model_info = manager.get_model_info(alias)
client = OpenAI(base_url=manager.endpoint, api_key=manager.api_key)
```

**வாழ்நாள் மேலாண்மை:**
- மேலாளர்கள் சரியான சுத்தம் செய்ய உலகளவில் சேமிக்கப்படுகிறார்கள்
- ஒவ்வொரு முகவரும் தனித்துவத்திற்காக வேறுபட்ட மாடலை பயன்படுத்த முடியும்
- தானியங்கி சேவை கண்டறிதல் மற்றும் இணைப்பு கையாளுதல்
- தோல்விகளில் மெல்லிய மீண்டும் முயற்சி செய்யும் தருக்கம்

இது முகவர் ஒருங்கிணைப்பு தொடங்குவதற்கு முன் சரியான தொடக்கத்தை உறுதிப்படுத்துகிறது.

**குறிப்பு:** https://github.com/microsoft/Foundry-Local/tree/main/sdk/python/foundry_local


In [18]:
import time

# Environment configuration
PRIMARY_ALIAS = os.getenv('AGENT_MODEL_PRIMARY', os.getenv('FOUNDRY_LOCAL_ALIAS', 'phi-4-mini'))
EDITOR_ALIAS = os.getenv('AGENT_MODEL_EDITOR', PRIMARY_ALIAS)

# Store managers globally for proper lifecycle management
primary_manager = None
editor_manager = None

def init_model(alias: str, max_retries: int = 3):
    """Initialize Foundry Local manager with retry logic.
    
    Args:
        alias: Model alias to initialize
        max_retries: Number of retry attempts with exponential backoff
    
    Returns:
        Tuple of (manager, client, model_id, endpoint)
    """
    delay = 2.0
    last_err = None
    
    for attempt in range(1, max_retries + 1):
        try:
            print(f"[Init] Starting Foundry Local for '{alias}' (attempt {attempt}/{max_retries})...")
            
            # Initialize manager - this starts the service and loads the model
            manager = FoundryLocalManager(alias)
            
            # Get model info to retrieve the actual model ID
            model_info = manager.get_model_info(alias)
            model_id = model_info.id
            
            # Create OpenAI client with manager's endpoint
            client = OpenAI(
                base_url=manager.endpoint,
                api_key=manager.api_key or 'not-needed'
            )
            
            # Verify the connection with a simple test
            models = client.models.list()
            print(f"[OK] Initialized '{alias}' -> {model_id} at {manager.endpoint}")
            
            return manager, client, model_id, manager.endpoint
            
        except Exception as e:
            last_err = e
            if attempt < max_retries:
                print(f"[Retry {attempt}/{max_retries}] Failed to init '{alias}': {e}")
                print(f"[Retry] Waiting {delay:.1f}s before retry...")
                time.sleep(delay)
                delay *= 2
            else:
                print(f"[ERROR] Failed to initialize '{alias}' after {max_retries} attempts")
    
    raise RuntimeError(f"Failed to initialize '{alias}' after {max_retries} attempts: {last_err}")

# Initialize primary model (for researcher)
print(f"\n{'='*80}")
print(f"Initializing Primary Model: {PRIMARY_ALIAS}")
print('='*80)
primary_manager, primary_client, PRIMARY_MODEL_ID, primary_endpoint = init_model(PRIMARY_ALIAS)

# Initialize editor model (may be same as primary)
if EDITOR_ALIAS != PRIMARY_ALIAS:
    print(f"\n{'='*80}")
    print(f"Initializing Editor Model: {EDITOR_ALIAS}")
    print('='*80)
    editor_manager, editor_client, EDITOR_MODEL_ID, editor_endpoint = init_model(EDITOR_ALIAS)
else:
    print(f"\n[Info] Editor using same model as primary")
    editor_manager = primary_manager
    editor_client, EDITOR_MODEL_ID = primary_client, PRIMARY_MODEL_ID
    editor_endpoint = primary_endpoint

print(f"\n{'='*80}")
print(f"[Configuration Summary]")
print('='*80)
print(f"  Primary Agent:")
print(f"    - Alias: {PRIMARY_ALIAS}")
print(f"    - Model: {PRIMARY_MODEL_ID}")
print(f"    - Endpoint: {primary_endpoint}")
print(f"\n  Editor Agent:")
print(f"    - Alias: {EDITOR_ALIAS}")
print(f"    - Model: {EDITOR_MODEL_ID}")
print(f"    - Endpoint: {editor_endpoint}")
print('='*80)



Initializing Primary Model: phi-4-mini
[Init] Starting Foundry Local for 'phi-4-mini' (attempt 1/3)...
[OK] Initialized 'phi-4-mini' -> Phi-4-mini-instruct-cuda-gpu:4 at http://127.0.0.1:59959/v1

Initializing Editor Model: gpt-oss-20b
[Init] Starting Foundry Local for 'gpt-oss-20b' (attempt 1/3)...
[OK] Initialized 'gpt-oss-20b' -> gpt-oss-20b-cuda-gpu:1 at http://127.0.0.1:59959/v1

[Configuration Summary]
  Primary Agent:
    - Alias: phi-4-mini
    - Model: Phi-4-mini-instruct-cuda-gpu:4
    - Endpoint: http://127.0.0.1:59959/v1

  Editor Agent:
    - Alias: gpt-oss-20b
    - Model: gpt-oss-20b-cuda-gpu:1
    - Endpoint: http://127.0.0.1:59959/v1


### விளக்கம்: Agent & Memory Classes
`AgentMsg` என்ற எளிய memory entries-ஐ வரையறுக்கிறது மற்றும் `Agent` encapsulating:
- **System role** - Agent-இன் தன்மை மற்றும் வழிகாட்டுதல்கள்
- **Message history** - உரையாடல் சூழலை பராமரிக்கிறது
- **act() method** - சரியான பிழை கையாளுதலுடன் செயல்களை நிறைவேற்றுகிறது

Agent பல்வேறு மாதிரிகளை (primary vs editor) பயன்படுத்த முடியும் மற்றும் ஒவ்வொரு agent-க்கும் தனித்துவமான சூழலை பராமரிக்கிறது. இந்த முறைமைகள் கீழ்கண்டவற்றை சாத்தியமாக்குகிறது:
- செயல்களில் memory persistence
- ஒவ்வொரு agent-க்கும் மாதிரிகளை சுலபமாக ஒதுக்குதல்
- பிழைகளை தனிமைப்படுத்தல் மற்றும் மீட்பு
- சுலபமான சங்கிலி மற்றும் ஒருங்கிணைப்பு


In [19]:
@dataclass
class AgentMsg:
    role: str
    content: str

@dataclass
class Agent:
    name: str
    system: str
    client: OpenAI = None  # Allow per-agent client assignment
    model_id: str = None   # Allow per-agent model
    memory: List[AgentMsg] = field(default_factory=list)

    def _history(self):
        """Return chat history in OpenAI messages format including system + memory."""
        msgs = [{'role': 'system', 'content': self.system}]
        for m in self.memory[-6:]:  # Keep last 6 messages to avoid context overflow
            msgs.append({'role': m.role, 'content': m.content})
        return msgs

    def act(self, prompt: str, temperature: float = 0.4, max_tokens: int = 300):
        """Send a prompt, store user + assistant messages in memory, and return assistant text.
        
        Args:
            prompt: User input/task for the agent
            temperature: Sampling temperature (0.0-1.0)
            max_tokens: Maximum tokens to generate
        
        Returns:
            Assistant response text
        """
        # Use agent-specific client/model or fall back to primary
        client_to_use = self.client or primary_client
        model_to_use = self.model_id or PRIMARY_MODEL_ID
        
        self.memory.append(AgentMsg('user', prompt))
        
        try:
            # Build messages including system prompt and history
            messages = self._history() + [{'role': 'user', 'content': prompt}]
            
            resp = client_to_use.chat.completions.create(
                model=model_to_use,
                messages=messages,
                max_tokens=max_tokens,
                temperature=temperature,
            )
            
            # Validate response
            if not resp.choices:
                raise RuntimeError("No completion choices returned")
            
            out = resp.choices[0].message.content or ""
            
            if not out:
                raise RuntimeError("Empty response content")
            
        except Exception as e:
            out = f"[ERROR:{self.name}] {type(e).__name__}: {str(e)}"
            print(f"[Agent Error] {self.name}: {type(e).__name__}: {str(e)}")
        
        self.memory.append(AgentMsg('assistant', out))
        return out

print("[INFO] Agent classes initialized with Foundry SDK support")
print(f"[INFO] Using OpenAI SDK version: {OpenAI.__module__}")


[INFO] Agent classes initialized with Foundry SDK support
[INFO] Using OpenAI SDK version: openai


### விளக்கம்: ஒர்கெஸ்ட்ரேட்டட் பைப்லைன்
இரு சிறப்பு முகவர்களை உருவாக்குகிறது:
- **ஆராய்ச்சியாளர்**: முதன்மை மாடலைப் பயன்படுத்தி, உண்மையான தகவல்களை சேகரிக்கிறது
- **தொகுப்பாளர்**: தனி மாடலை (கட்டமைக்கப்பட்டால்) பயன்படுத்த முடியும், தகவல்களை மேம்படுத்தி மறுபதிப்பாக்கம் செய்கிறது

`pipeline` செயல்பாடு:
1. ஆராய்ச்சியாளர் மூல தகவல்களை சேகரிக்கிறார்
2. தொகுப்பாளர் அதை நிர்வாகத்துக்கு உகந்த வெளியீடாக மேம்படுத்துகிறார்
3. இடைநிலை மற்றும் இறுதி முடிவுகளை திருப்பி வழங்குகிறது

இந்த முறைமையானது:
- மாடல் சிறப்பு செயல்பாடுகளை (வேறு வேறு பங்குகளுக்கு வேறு மாடல்கள்) சாத்தியமாக்குகிறது
- பல நிலை செயலாக்கத்தின் மூலம் தரத்தை மேம்படுத்துகிறது
- தகவல் மாற்றத்தின் கண்காணிப்புத் திறனை வழங்குகிறது
- மேலும் முகவர்கள் அல்லது இணை செயலாக்கத்துக்கு எளிதில் விரிவாக்கம் செய்ய உதவுகிறது


In [ ]:
# Create specialized agents with optional model assignment
researcher = Agent(
    name='Researcher',
    system='You collect concise factual bullet points.',
    client=primary_client,
    model_id=PRIMARY_MODEL_ID
)

editor = Agent(
    name='Editor',
    system='You rewrite content for clarity and an executive, action-focused tone.',
    client=editor_client,
    model_id=EDITOR_MODEL_ID
)

def pipeline(q: str, verbose: bool = True):
    """Execute multi-agent pipeline: Researcher -> Editor.
    
    Args:
        q: User question/task
        verbose: Print intermediate outputs
    
    Returns:
        Dictionary with research, final outputs, and metadata
    """
    if verbose:
        print(f"[Pipeline] Question: {q}\n")
    
    # Stage 1: Research
    if verbose:
        print("[Stage 1: Research]")
    research = researcher.act(q)
    if verbose:
        print(f"Output: {research[:200]}...\n")
    
    # Stage 2: Editorial refinement
    if verbose:
        print("[Stage 2: Editorial Refinement]")
    rewrite = editor.act(
        f"Rewrite professionally with a 1-sentence executive summary first. "
        f"Improve clarity, keep bullet structure if present. Source:\n{research}"
    )
    if verbose:
        print(f"Output: {rewrite[:200]}...\n")
    
    return {
        'question': q,
        'research': research,
        'final': rewrite,
        'models': {
            'researcher': PRIMARY_MODEL_ID,
            'editor': EDITOR_MODEL_ID
        }
    }

# Execute sample pipeline
print("="*80)
result = pipeline('Explain why edge AI matters for compliance and latency.')
print("="*80)
print("\n[FINAL OUTPUT]")
print(result['final'])
print("\n[METADATA]")
print(f"Models used: {result['models']}")
result

[Pipeline] Question: Explain why edge AI matters for compliance and latency.

[Stage 1: Research]
Output: - **Data Sovereignty**: Edge AI allows data to be processed locally, which can help organizations comply with regional data protection regulations by keeping sensitive information within the borders o...

[Stage 2: Editorial Refinement]


### விளக்கம்: குழாய் செயல்பாடு மற்றும் முடிவுகள்
கோப்புரம் + தாமதம் தொடர்பான கேள்வியில் பல முகவர்களின் குழாய் செயல்பாட்டை செயல்படுத்தி காட்டுகிறது:
- பல நிலை தகவல் மாற்றம்
- முகவர்களின் நிபுணத்துவம் மற்றும் ஒத்துழைப்பு
- சீரமைப்பின் மூலம் வெளியீட்டு தரத்தை மேம்படுத்தல்
- கண்காணிப்பு (இடைநிலை மற்றும் இறுதி வெளியீடுகள் இரண்டும் பாதுகாக்கப்பட்டவை)

**முடிவு அமைப்பு:**
- `question` - முதன்மை பயனர் கேள்வி
- `research` - மூல ஆராய்ச்சி வெளியீடு (உண்மையான புள்ளிகள்)
- `final` - சீரமைக்கப்பட்ட நிர்வாக சுருக்கம்
- `models` - ஒவ்வொரு நிலைக்கும் எந்த மாதிரிகள் பயன்படுத்தப்பட்டன

**விரிவாக்கக் கருத்துகள்:**
1. தர மதிப்பீட்டிற்கான ஒரு விமர்சகர் முகவரை சேர்க்கவும்
2. வேறு அம்சங்களுக்கு இணை ஆராய்ச்சி முகவர்களை செயல்படுத்தவும்
3. உண்மைகளை சரிபார்க்க ஒரு சரிபார்ப்பாளர் முகவரை சேர்க்கவும்
4. வேறு சிக்கல்தன்மை நிலைகளுக்கு வேறு மாதிரிகளை பயன்படுத்தவும்
5. மீள்நோக்கு மடல்களை செயல்படுத்தி மீள்மேம்பாட்டை மேற்கொள்ளவும்


### மேம்பட்டது: தனிப்பயன் முகவர் அமைப்பு

முகவரின் செயல்பாட்டை ஆரம்ப செல்களை இயக்குவதற்கு முன் சூழல் மாறிகளை மாற்றுவதன் மூலம் தனிப்பயனாக்க முயற்சிக்கவும்:

**கிடைக்கக்கூடிய மாதிரிகள்:**
- டெர்மினலில் `foundry model ls` பயன்படுத்தி அனைத்து கிடைக்கக்கூடிய மாதிரிகளையும் பார்க்கவும்
- உதாரணங்கள்: phi-4-mini, phi-3.5-mini, qwen2.5-7b, llama-3.2-3b, போன்றவை.


In [ ]:
# Example: Use different models for different agents
# Uncomment and modify as needed:

# import os
# os.environ['AGENT_MODEL_PRIMARY'] = 'phi-4-mini'      # Fast, good for research
# os.environ['AGENT_MODEL_EDITOR'] = 'qwen2.5-7b'       # Higher quality for editing

# Then restart the kernel and re-run all cells

# Test with different questions
test_questions = [
    "What are 3 key benefits of using small language models?",
    "How does RAG improve AI accuracy?",
    "Why is local inference important for privacy?"
]

print("Testing pipeline with multiple questions:\n")
for i, q in enumerate(test_questions, 1):
    print(f"\n{'='*80}")
    print(f"Question {i}: {q}")
    print('='*80)
    r = pipeline(q, verbose=False)
    print(f"\n[FINAL]: {r['final'][:300]}...")
    print(f"[Models]: Researcher={r['models']['researcher']}, Editor={r['models']['editor']}")


Testing pipeline with multiple questions:


Question 1: What are 3 key benefits of using small language models?

[FINAL]: <|channel|>analysis<|message|>The user wants a rewrite of the entire block of text. The rewrite should be professional, include a one-sentence executive summary first, improve clarity, keep bullet structure if present. The user has provided a large amount of text. The user wants a rewrite of that te...
[Models]: Researcher=Phi-4-mini-instruct-cuda-gpu:4, Editor=gpt-oss-20b-cuda-gpu:1

Question 2: How does RAG improve AI accuracy?

[FINAL]: <|channel|>final<|message|>**RAG (Retrieval‑Augmented Generation) empowers AI to produce highly accurate, contextually relevant responses by combining a retrieval system with a large language model (LLM).**<|return|>...
[Models]: Researcher=Phi-4-mini-instruct-cuda-gpu:4, Editor=gpt-oss-20b-cuda-gpu:1

Question 3: Why is local inference important for privacy?

[FINAL]: <|channel|>final<|message|>**Local inference—processing data d


---

**அறிவிப்பு**:  
இந்த ஆவணம் [Co-op Translator](https://github.com/Azure/co-op-translator) என்ற AI மொழிபெயர்ப்பு சேவையை பயன்படுத்தி மொழிபெயர்க்கப்பட்டுள்ளது. நாங்கள் துல்லியத்திற்காக முயற்சிக்கிறோம், ஆனால் தானியங்கி மொழிபெயர்ப்புகளில் பிழைகள் அல்லது தவறுகள் இருக்கக்கூடும் என்பதை தயவுசெய்து கவனத்தில் கொள்ளவும். அதன் சொந்த மொழியில் உள்ள மூல ஆவணம் அதிகாரப்பூர்வ ஆதாரமாக கருதப்பட வேண்டும். முக்கியமான தகவல்களுக்கு, தொழில்முறை மனித மொழிபெயர்ப்பு பரிந்துரைக்கப்படுகிறது. இந்த மொழிபெயர்ப்பைப் பயன்படுத்துவதால் ஏற்படும் எந்த தவறான புரிதல்களுக்கும் அல்லது தவறான விளக்கங்களுக்கும் நாங்கள் பொறுப்பல்ல.
